# Intégration des données dans un data warehouse
# BUT SD 2
##### Par: Tim Touviot, Éloïse Serrand, Julien Renoult, Dalyop Yop
###### 24/11/2023 - 04/12/2023

# Initialisation

Dans cette partie, nous allons importer les librairies nécessaires au bon fonctionnement du programme. Ensuite, nous allons mettre en place le chemin permettant de récupérer les données préparées précédemment. Enfin, nous allons importer ces données pour pouvoir les insérer dans la base de données.

In [2]:
import pandas as pd
import os
import datetime
import mysql.connector
os.getcwd()

'/home/serrand222/Documents/BUT2/SAE_INT'

## Création des chemins

Ces deux chemins vont être créés pour récupérer les données préparées qui sont conservées dans un dossier que l'on va préciser.

In [5]:
# Adapter les chemins si nécessaire :
chemin_meteo = ""
chemin_pollution = ""

## Récupération des données pour l'insertion

On va maintenant importer les données préparées dans deux trames nommées _donnees_pollution_ et _donnees_meteo_.

In [6]:
# Récupération des données de pollution
donnees_pollution = pd.read_csv(chemin_pollution+"pollution.csv")

# Vérification du succès de l'opération
print(donnees_pollution)

# Récupération des données météorologique
donnees_meteo = pd.read_csv(chemin_meteo+"meteo.csv")

# Vérification du succès de l'opération
print(donnees_meteo)

         Unnamed: 0             DateTime     Station  Type  Unite     Valeur
0            151032  2017-03-25 01:00:00     firvale   NO2    ppb  20.160400
1            151033  2017-03-25 02:00:00     firvale   NO2    ppb  19.264313
2            151034  2017-03-25 03:00:00     firvale   NO2    ppb  19.824310
3            151035  2017-03-25 04:00:00     firvale   NO2    ppb  21.840393
4            151036  2017-03-25 05:00:00     firvale   NO2    ppb  20.832306
...             ...                  ...         ...   ...    ...        ...
1781188      198893  2022-09-09 06:00:00  the wicker  PM25  ug/m3   0.000000
1781189      198894  2022-09-09 07:00:00  the wicker  PM25  ug/m3   0.000000
1781190      198895  2022-09-09 08:00:00  the wicker  PM25  ug/m3   0.000000
1781191      198896  2022-09-09 09:00:00  the wicker  PM25  ug/m3   0.000000
1781192      198897  2022-09-09 10:00:00  the wicker  PM25  ug/m3   0.000000

[1781193 rows x 6 columns]
         Unnamed: 0             DateTime        

# Récupération des types et unités de mesures

Dans cette partie, pour la future table TYPE à créer, on a besoin d'avoir les types avec leurs unités de mesure. Pour ce faire, on va faire pour chacune des trames qui sont _donnees_pollution_ et _donnees_meteo_, de trouver ces valeurs, les concaténer et supprimer les doublons pour avoir des lignes de données uniques pour la table TYPE.

## Dans le cas des données de pollutions

On va d'abord récupérer les deux colonnes d'intérêts qui sont __Type__ et __Unite__. Après ça, nous allons supprimer les doublons pour n'avoir que des lignes uniques.

In [7]:
# Récupérer les colonnes d'intérêts
type_unite_P = donnees_pollution[["Type", "Unite"]]

# Supprimer les doublons
type_unite_P = type_unite_P.drop_duplicates()

# Vérification du succès de l'opération
type_unite_P[["Type", "Unite"]]

Type  Unite
0             NO2    ppb
36520          NO    ppb
68688        PM10  ug/m3
99063        PM25  ug/m3
130180   Pressure    hPa
161297        SO2    ppb
251086       Temp  deg.C
780249      Ozone    ppb
1006196     PM2.5  ugm-3

## Dans le cas des données météo

C'est la même méthode pour les données météo ce qui permet aussi d'avoir des lignes uniques entre elles.

In [8]:
# Récupérer les colonnes d'intérêts
type_unite_M = donnees_meteo[["Type", "Unite"]]

# Supprimer les doublons
type_unite_M = type_unite_M.drop_duplicates()

# Vérification du succès de l'opération
type_unite_M[["Type", "Unite"]]

Type  Unite
0            DiffTemp 2-8 m   deg.C
189464      DiffTemp 8-22 m   deg.C
220076     Global Radiation    W/m2
260201  Max Wind Speed 10 m     m/s
290813  Max Wind Speed 24 m     m/s
321425             Pressure     hPa
352037                 Rain      mm
382648                 Temp   deg.C
575533  Wind Direction 10 m   deg.M
768224  Wind Direction 24 m   deg.M
798836      Wind Speed 10 m     m/s
991784      Wind Speed 24 m     m/s

## Concaténation et supprimer les doublons

Après avoir réussi à récupérer les types et unités de mesure, nous allons procéder à une concaténation des deux trames qui sont _type_unite_P_ et _type_unite_M_ pour avoir tous les types et unités de mesure. Après concaténation, nous allons d'abord modifier certaines unités, car elles ont la même signification sous une syntaxe différente (Cas du 'deg C' et 'deg.C'). Pour finir, on va finalement supprimer les doublons restants après la concaténation.

In [9]:
# Concaténation des deux dataframes
type_unite = pd.concat([type_unite_P, type_unite_M])

# Mettre la même unité pour deg.C
type_unite = type_unite.replace("deg C", "deg.C")

# Supprimer les doublons
type_unite = type_unite.drop_duplicates()

# Vérification du succès de l'opération
type_unite[["Type", "Unite"]]

Type  Unite
0                         NO2    ppb
36520                      NO    ppb
68688                    PM10  ug/m3
99063                    PM25  ug/m3
130180               Pressure    hPa
161297                    SO2    ppb
251086                   Temp  deg.C
780249                  Ozone    ppb
1006196                 PM2.5  ugm-3
0             DiffTemp 2-8 m   deg.C
189464       DiffTemp 8-22 m   deg.C
220076      Global Radiation    W/m2
260201   Max Wind Speed 10 m     m/s
290813   Max Wind Speed 24 m     m/s
321425              Pressure     hPa
352037                  Rain      mm
382648                  Temp   deg.C
575533   Wind Direction 10 m   deg.M
768224   Wind Direction 24 m   deg.M
798836       Wind Speed 10 m     m/s
991784       Wind Speed 24 m     m/s

# Création de la base de données

Pour ce faire, nous allons créer un fichier texte avec les informations ci-dessous :

user : utilisateur  
password : mot de passe  
host : hôte  
database : base de données   

On doit respecter ce format (avec les espaces et les retours à la ligne) pour pouvoir récupérer les informations de connexions à la base de données. Cela permet de créer le dictionnaire de config_connexion pour la connexion à la base de données par mySQL.

```python
config_connexion = {
    'user' : 'utilisateur',
    'password' : 'mot de passe',
    'host' : 'hôte',
    'database' : 'base de données'
}
```

On nommera ce fichier Donnees.txt en sauvegardant.

In [19]:
# Adapter le chemin si nécessaire
chemin_config = ""

# Récupération des informations nécessaires à la connexion
bloc = open(chemin_config+"Donnees.txt").read()

# Association des informations par leurs valeurs
liste = bloc.split("\n")

config_connexion = {}

for elt in liste:
    if elt != "":
        liste_2 = elt.split(" : ")
        config_connexion[liste_2[0]] = liste_2[1]

#print(config_connexion)

## Création des paramètres de connexion

Après avoir récupéré les différents éléments nécessaires à la connexion, nous allons les mettre dans des variables explicites permettant la connexion à la base.

In [20]:
# Paramètres de connexion
DBNAME = config_connexion["database"]
DBHOST = config_connexion["host"]
DBUSER = config_connexion["user"]
DBPASS = config_connexion["password"]

## Fonction d'insertion

Cette fonction permet l'insertion des données par les requêtes de type 'INSERT' avec l'utilisation du _commit_ pour obliger à la base de données de les sauvegarder.

In [21]:
def mysqlinsertdata(sql, data):
    """
    Fonction qui permet l'insertion des données par les requêtes de type 'INSERT'. 
    Les arguments sont :
    
    - sql : requête SQL
    - data : données à utiliser
    
    """
    try :
        connection = mysql.connector.connect(
            host        = DBHOST,
            user        = DBUSER,
            password    = DBPASS,
            database    = DBNAME)
        cursor = connection.cursor()
        cursor.execute(sql, data)
        connection.commit()
        connection.close()
    except Exception as e :
        print("Error :", e)

## Fonction de requête

Cette fonction permet d'exécuter toutes les autres types de requêtes ('CREATE', 'SELECT', 'DROP',...).

In [22]:
def mysqlrequest(sql, data) :
    """
    Fonction qui permet l'insertion des données par tous autres types de requêtes avec comme arguments' :
    
    - sql : requête SQL
    - data : données à utiliser
    
    """
    result = None
    try :
        connection = mysql.connector.connect(
            host        = DBHOST,
            user        = DBUSER,
            password    = DBPASS,
            database    = DBNAME)
        cursor = connection.cursor()
        cursor.execute(sql, data)
        result = cursor.fetchall()
        connection.close()
    except Exception as e :
        print("Error :", e)
    return result

## Fonction d'observation de données

Cette fonction permet de rapidement d'observer les données d'une table pour observer le succès de l'insertion des données.

In [23]:
def mysqldonnee(nom_table, nb=0) :
    """
    Fonction qui permet d'observer des données en précisant 
    le nom de la table (nom_table) et le nombre d'observations à observer (nb).
    Le nombre d'observations à prendre n'est pas obligatoire et vous pouvez passer cette option pour 
    avoir toutes les lignes de données si bien sûr la table ne possède pas beaucoup d'enregistrement.
    """
    result = None
    try :
        connection = mysql.connector.connect(
            host        = DBHOST,
            user        = DBUSER,
            password    = DBPASS,
            database    = DBNAME)
        cursor = connection.cursor()
        if nb !=0:
            cursor.execute("SELECT * FROM "+nom_table+" LIMIT "+str(nb)+";", ())
        else:
            cursor.execute("SELECT * FROM "+nom_table+" ;", ())
            
        result = cursor.fetchall()
        connection.close()
    except Exception as e :
        print("Error :", e)
    return result

# Création de la base de données

Ici, on va procéder à la création des différentes tables permettant à la suite de faire l'insertion des données.

In [12]:
req_cre = [
    # Cas de la table MESURE
    """CREATE TABLE MESURE (
     date DATETIME NOT NULL,
     station CHAR(50) NOT NULL,
     type char(50) NOT NULL,
     valeur DECIMAL(14, 10) NOT NULL,
     CONSTRAINT IDMESURE PRIMARY KEY (date, station, type));""",
    
    # Cas de la table TYPE
    """CREATE TABLE TYPE (
     type CHAR(50) NOT NULL,
     unite CHAR(10) NOT NULL,
     CONSTRAINT IDTYPE PRIMARY KEY (type));"""
]

# La partie d'exécution des requêtes de création de table
for req in req_cre :
    mysqlrequest(req, ())

Error : 1050 (42S01): Table 'MESURE' already exists
Error : 1050 (42S01): Table 'TYPE' already exists


# Insertion dans la base de données

Ici, nous allons procéder à l'insertion des données des tables MESURE et TYPE. Pour la table des MESURES, nous allons procéder à deux boucles for permettant d'une part l'insertion des données pollution et d'autre part l'insertion des données météorologiques. Pour la table des TYPE, nous allons utiliser la trame __type_unite__ permettant d'avoir les types et unités de mesure.

## Insertion des données dans la table TYPE

In [13]:
# Liste de requête d'INSERT pour les deux tables
req_ins = ["INSERT INTO MESURE(date, station, type, valeur) VALUES (%s, %s, %s, %s);",
          "INSERT INTO TYPE(type, unite) VALUES (%s, %s)"]

# Insertion des types et unités de mesures
for elt in type_unite.to_records(index=False):
    mysqlinsertdata(req_ins[1], list(elt))

# Vérification du succès de l'opération
print(mysqldonnee("TYPE"))

Error : 1062 (23000): Duplicate entry 'Pressure' for key 'PRIMARY'
Error : 1062 (23000): Duplicate entry 'Temp' for key 'PRIMARY'
[('DiffTemp 2-8 m', 'deg.C'), ('DiffTemp 8-22 m', 'deg.C'), ('Global Radiation', 'W/m2'), ('Max Wind Speed 10 m', 'm/s'), ('Max Wind Speed 24 m', 'm/s'), ('NO', 'ppb'), ('NO2', 'ppb'), ('Ozone', 'ppb'), ('PM10', 'ug/m3'), ('PM2.5', 'ugm-3'), ('PM25', 'ug/m3'), ('Pressure', 'hPa'), ('Rain', 'mm'), ('SO2', 'ppb'), ('Temp', 'deg.C'), ('Wind Direction 10 m', 'deg.M'), ('Wind Direction 24 m', 'deg.M'), ('Wind Speed 10 m', 'm/s'), ('Wind Speed 24 m', 'm/s')]


## Insertion des données dans la table MESURE

### Dans le cas des données sur la pollution

In [30]:
# Insertion des données sur la pollution dans la table MESURE
for elt in donnees_pollution.to_records(index = False):
    date = datetime.datetime.strptime(elt[1], "%Y-%m-%d %H:%M:%S")
    mysqlinsertdata(req_ins[0], [date, elt[2], elt[3], elt[5]])

# Vérification du succès de l'opération
print(mysqldonnee("MESURE", 10))

[(datetime.datetime(2000, 2, 1, 0, 15), 'groundhog2', 'SO2', Decimal('0E-10')), (datetime.datetime(2000, 2, 1, 0, 30), 'groundhog2', 'SO2', Decimal('0E-10')), (datetime.datetime(2000, 2, 1, 1, 0), 'groundhog2', 'PM10', Decimal('9.2625000000')), (datetime.datetime(2000, 2, 1, 1, 30), 'groundhog2', 'SO2', Decimal('0E-10')), (datetime.datetime(2000, 2, 1, 1, 45), 'groundhog2', 'SO2', Decimal('0E-10')), (datetime.datetime(2000, 2, 1, 2, 0), 'groundhog2', 'PM10', Decimal('7.1500000000')), (datetime.datetime(2000, 2, 1, 2, 0), 'groundhog2', 'SO2', Decimal('0E-10')), (datetime.datetime(2000, 2, 1, 2, 15), 'groundhog2', 'SO2', Decimal('0E-10')), (datetime.datetime(2000, 2, 1, 2, 30), 'groundhog2', 'SO2', Decimal('0E-10')), (datetime.datetime(2000, 2, 1, 2, 45), 'groundhog2', 'SO2', Decimal('0E-10'))]


In [25]:
# Insertion des données météorologiques dans la table MESURE
for elt in donnees_meteo.to_records(index = False):
    date = datetime.datetime.strptime(elt[1], "%Y-%m-%d %H:%M:%S")
    mysqlinsertdata(req_ins[0], [date, elt[2], elt[3], elt[5]])
    
# Vérification du succès de l'opération
print(mysqldonnee("MESURE", 10))

[(datetime.datetime(2000, 1, 1, 1, 0), 'athletic stadium', 'DiffTemp 2-8 m', Decimal('-0.0740000000')), (datetime.datetime(2000, 1, 1, 1, 0), 'athletic stadium', 'Temp', Decimal('14.8799900000')), (datetime.datetime(2000, 1, 1, 1, 0), 'athletic stadium', 'Wind Direction 10 m', Decimal('222.0000000000')), (datetime.datetime(2000, 1, 1, 1, 0), 'athletic stadium', 'Wind Speed 10 m', Decimal('2.5149990000')), (datetime.datetime(2000, 1, 1, 2, 0), 'athletic stadium', 'DiffTemp 2-8 m', Decimal('-0.1660000000')), (datetime.datetime(2000, 1, 1, 2, 0), 'athletic stadium', 'Temp', Decimal('15.3399960000')), (datetime.datetime(2000, 1, 1, 2, 0), 'athletic stadium', 'Wind Direction 10 m', Decimal('230.8999020000')), (datetime.datetime(2000, 1, 1, 2, 0), 'athletic stadium', 'Wind Speed 10 m', Decimal('2.5989990000')), (datetime.datetime(2000, 1, 1, 3, 0), 'athletic stadium', 'DiffTemp 2-8 m', Decimal('-0.1830000000')), (datetime.datetime(2000, 1, 1, 3, 0), 'athletic stadium', 'Temp', Decimal('18.16